## Q1 Define function description

In [45]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [82]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather data for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to query the weather data"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

TODO3: city

In [ ]:
from groq import Groq
import os
from dotenv import load_dotenv

# Cargar .env
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [89]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

In [90]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data for a specific city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city to query the weather data'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [91]:
client = Groq(api_key=api_key)

In [93]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

## Q2. Adding another tool

def set_weather(city: str, temp: float) -> None:
  city = city.strip().lower()
  known_weather_data[city] = temp
  return 'OK'

Answer:

In [96]:
set_weather_tool = {
  "type": "function",
  "name": "set_weather",
  "description": "Sets the temperature for a specified city or region",
  "parameters": {
    "type": "object",
    "properties": {
      "city": {
        "type": "string",
        "description": "The name of the city, e.g. New York"
      },
      "temp": {
        "type": "number",
        "description": "The temperature to associate with the city, e.g. 20"
      }
    },
    "required": ["city", "temp"],
    "additionalProperties": False
  }
}

## Q3. Install FastMCP

What's the version of FastMCP you installed?


In [99]:
import fastmcp

fastmcp.__version__

'2.10.5'

## Q4. Simple MCP Server

In [100]:
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

What do you see in the output?
Starting MCP server 'WeatherBot ' with transport 'stdio'

## Q5. Protocol

There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Let's ask the temperature in Berlin:

{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}

What did you get in response?

{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": { "city": "Berlin" } } }

Answer:
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

## Q6. Client

We typically don't interact with the server by copy-pasting commands in the terminal.

In practice, we use an MCP Client. Let's implement it.

Use the client to get the list of available tools of our script. How does the result look like?

In [104]:
from fastmcp import Client
import server
import json

async def main():
  async with Client(server.mcp) as mcp_client:
    return await mcp_client.list_tools()

tool_list = await main()

tool_list

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None),
 Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'T